In [1]:
import torch

In [5]:
import requests
from pathlib import Path
from zipfile import ZipFile

data_path = Path("data/pizza_steak_sushi")

train_dir = data_path / "train"
test_dir = data_path / "test"

if data_path.is_dir():
    print("Folder sudah ada")
else: 
    with open("data/pizza_steak_sushi.zip", "wb") as f:
        res = requests.get(
            "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"
        )
        f.write(res.content)

    with ZipFile("data/pizza_steak_sushi.zip","r") as zf:
        zf.extractall(data_path)

Folder sudah ada


In [16]:
from torchvision.transforms import v2

train_transforms = v2.Compose([
        v2.ToImage(),
        v2.Resize(size=(64,64)),
        v2.TrivialAugmentWide(num_magnitude_bins=31),
        v2.ToDtype(torch.float32,scale=True)
    ])

test_transforms = v2.Compose([
        v2.ToImage(),
        v2.Resize(size=(64,64)),
        v2.ToDtype(torch.float32,scale=True)
    ])

In [13]:
%%writefile going_modular/data_setup.py

from torchvision.datasets import ImageFolder
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import os

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transforms: v2.Compose,
    test_transforms: v2.Compose,
    batch_size: int = BATCH_SIZE,
    num_workers: int = NUM_WORKERS,
):
    train_data = ImageFolder(root=train_dir,transform=train_transforms)
    test_data = ImageFolder(root=test_dir, transform=test_transforms)
    class_names = train_data.classes

    train_dataloader = DataLoader(
        dataset=train_data, batch_size=batch_size, num_workers=num_workers,shuffle=True,pin_memory=True
    )
    test_dataloader = DataLoader(
        dataset=test_data, batch_size=batch_size, num_workers=num_workers,pin_memory=True
    )

    return train_dataloader,test_dataloader,class_names

Writing going_modular/data_setup.py


In [22]:
import os
from going_modular import data_setup

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

train_dir = data_path / "train"
test_dir = data_path / "test"

train_dataloader,test_dataloader,class_names = data_setup.create_dataloaders(train_dir=train_dir,test_dir=test_dir,train_transforms=train_transforms,test_transforms=test_transforms,batch_size=BATCH_SIZE,num_workers=NUM_WORKERS)

train_dataloader,test_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x22c318f0760>,
 ['pizza', 'steak', 'sushi'])

In [21]:
train_dataloader.batch_size

32

In [23]:
%%writefile going_modular/model_builder.py

from torch import nn
class TinyVGG(nn.Module):
    def __init__(self,input_shape:int,hidden_units:int,output_shape:int):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_units, 3, 1, 1),
            nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units * 2, 3, 1, 1),
            nn.Conv2d(hidden_units * 2, hidden_units * 2, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.conv_block_3 = nn.Sequential(
            nn.Conv2d(hidden_units * 2, hidden_units, 3, 1, 1),
            nn.Conv2d(hidden_units, hidden_units, 3, 1, 1),
            nn.MaxPool2d(2),
        )

        self.classifer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units * 8 * 8, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16,output_shape)
        )

    def forward(self,x):
        return self.classifer(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))

Writing going_modular/model_builder.py


In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
from going_modular.model_builder import TinyVGG

model = TinyVGG(input_shape=3,hidden_units=32,output_shape=len(class_names)).to(device)
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2048, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_featu

In [25]:
from torchinfo import summary

summary(model,input_size=(32,3,64,64))

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 32, 32, 32]          --
│    └─Conv2d: 2-1                       [32, 32, 64, 64]          896
│    └─Conv2d: 2-2                       [32, 32, 64, 64]          9,248
│    └─MaxPool2d: 2-3                    [32, 32, 32, 32]          --
├─Sequential: 1-2                        [32, 64, 16, 16]          --
│    └─Conv2d: 2-4                       [32, 64, 32, 32]          18,496
│    └─Conv2d: 2-5                       [32, 64, 32, 32]          36,928
│    └─MaxPool2d: 2-6                    [32, 64, 16, 16]          --
├─Sequential: 1-3                        [32, 32, 8, 8]            --
│    └─Conv2d: 2-7                       [32, 32, 16, 16]          18,464
│    └─Conv2d: 2-8                       [32, 32, 16, 16]          9,248
│    └─MaxPool2d: 2-9                    [32, 32, 8, 8]           

In [28]:
img,label = next(iter(train_dataloader))

img[0],label[0]

(tensor([[[0.2000, 0.7569, 0.7843,  ..., 0.3216, 0.3176, 0.3059],
          [0.7961, 0.7686, 0.7882,  ..., 0.3098, 0.3059, 0.2980],
          [0.7882, 0.7804, 0.7882,  ..., 0.2980, 0.3020, 0.2824],
          ...,
          [0.6667, 0.6706, 0.6863,  ..., 0.4510, 0.4471, 0.5529],
          [0.6784, 0.6824, 0.6902,  ..., 0.4980, 0.5765, 0.6588],
          [0.7137, 0.7098, 0.7176,  ..., 0.5333, 0.5412, 0.6196]],
 
         [[0.1608, 0.1647, 0.1451,  ..., 0.4745, 0.5216, 0.5686],
          [0.1647, 0.1686, 0.1529,  ..., 0.5294, 0.5569, 0.5608],
          [0.1686, 0.1686, 0.1569,  ..., 0.5490, 0.5451, 0.4902],
          ...,
          [0.7333, 0.7373, 0.7490,  ..., 0.4863, 0.4824, 0.5961],
          [0.7412, 0.7412, 0.7451,  ..., 0.5412, 0.6196, 0.7020],
          [0.7647, 0.7608, 0.7647,  ..., 0.5647, 0.5765, 0.6745]],
 
         [[0.1843, 0.1725, 0.1608,  ..., 0.5020, 0.5804, 0.6667],
          [0.1843, 0.1804, 0.1686,  ..., 0.5922, 0.6431, 0.6588],
          [0.1882, 0.1804, 0.1765,  ...,

In [31]:
model.eval(),

with torch.inference_mode():
    y_logits = model(img[0].unsqueeze(0).to(device))
    print(y_logits)

tensor([[ 0.1004, -0.0061, -0.1821]], device='cuda:0')
